## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
sys.path.append('../..')
import stober_synthesis_utils as stober

In [4]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = '2025_01_28_80nmRound1_Opt.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [5]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

In [6]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [7]:
sample_table_rmse = pd.read_csv('Sample_table_2025_01_28_80nmRound1_rmse.csv')
sample_table_apdist = pd.read_csv('Sample_table_2025_01_28_80nm_Round1_apdist.csv')

In [53]:
dilute_wells= ['B4', 'B5', 'B6', 'B8', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6']

In [56]:
sample_table_rmse['well'] = sample_table_apdist['well']
sample_table_rmse['dilute_well'] = dilute_wells
sample_table_apdist['dilute_well'] = dilute_wells

In [57]:
sample_table_rmse

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,9cff952d-2d28-47b2-bb0b-d49602fd36d2,90.614116,189.664150,250.625458,1169.096276,1.193493,5.033963,0.006237,0.0,1700.0,A1,B4
1,1,60e54cc2-4d8e-4d0e-aab8-390bef44dcf8,228.684063,270.377199,470.888258,730.050480,0.472911,12.704280,0.015444,896.0,804.0,A2,B5
2,2,bde1a208-9fee-499e-843b-7866aa600a77,24.224999,484.500007,666.348728,524.926265,4.464286,1.345792,0.001622,0.0,1700.0,A3,B6
3,3,8f8ed9a5-2ea3-440f-93bf-01d9f6cbc33d,484.500007,24.224999,273.731449,917.543544,0.223214,26.915840,0.032983,1321.0,379.0,A4,B8
4,4,f93ed4f1-94fb-4806-835e-4975da39c452,135.707419,373.352663,489.220082,701.719836,0.796915,7.539070,0.009161,345.0,1355.0,A5,C1
5,5,99049d8c-98af-4240-ad63-7784477d7c02,379.921139,133.905259,221.684161,964.489441,0.284657,21.106082,0.025954,1216.0,484.0,A6,C2
6,6,a35e929b-49fd-4bed-a211-70524348fe8e,24.224999,212.324890,226.751481,1236.698630,4.464286,1.345792,0.001673,0.0,1700.0,A7,C3
7,7,069a5c5d-4bfd-419e-a96e-36215855466d,211.666209,484.500007,538.140465,465.693319,0.510933,11.758873,0.014169,831.0,869.0,A8,C4
8,8,627048de-ccfc-4b9e-998f-c83de2ac353f,199.575596,135.113642,701.939108,663.371654,0.541887,11.087193,0.013380,779.0,921.0,B1,C5
9,9,9f8223fc-c3be-4ca7-b9c9-55fa9c4daa60,484.500007,246.977225,63.183338,905.339430,0.223214,26.915840,0.033145,1321.0,379.0,B3,C6


In [59]:
sample_table_apdist

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,7c4d74ad-5362-4981-b7a5-52fee1cd2e48,100.446867,431.840514,39.876055,1127.836564,1.076662,5.580210,0.006942,0.0,1700.0,A1,B4
1,1,2b778460-0ea0-4134-8256-cc0950b1838f,47.743600,169.627361,480.552179,1002.076860,2.265169,2.652341,0.003253,0.0,1700.0,A2,B5
2,2,e73de4ac-ae2f-40ac-adc9-6f4acfbb4678,297.684084,187.260579,659.409887,555.645450,0.363296,16.537497,0.019903,1082.0,618.0,A3,B6
3,3,6e8ce412-9cdc-41b3-b8ae-fa08acb207cf,165.119200,457.338778,723.993653,353.548369,0.654965,9.173007,0.010973,587.0,1113.0,A4,B8
4,4,ec102405-f168-46a3-935d-2d24fdaa5b1f,83.489946,282.278352,27.288292,1306.943410,1.295333,4.638188,0.005802,0.0,1700.0,A5,C1
5,5,7bf7296f-8eae-4228-a615-3d7b8c142d89,74.110835,34.513714,48.689469,1542.685982,1.459265,4.117142,0.005183,0.0,1700.0,A6,C2
6,6,7d38b9e2-e3d3-494a-b43a-2997b6d7bf4c,91.456437,456.318141,63.359785,1088.865637,1.182501,5.080757,0.006310,0.0,1700.0,A7,C3
7,7,2252e0ad-3f6b-4234-9603-2420244f1080,230.246387,172.913187,526.767735,770.072691,0.469703,12.791073,0.015545,902.0,798.0,A8,C4
8,8,bf8528f0-4b3a-429d-bb11-da3c7d2c66f7,447.470939,365.846071,91.428678,795.254312,0.241686,24.858733,0.030505,1289.0,411.0,B1,C5
9,9,4c109404-def8-474c-a617-783ec03ec895,43.538614,94.802899,335.081280,1226.577207,2.483940,2.418738,0.002996,0.0,1700.0,B3,C6


In [60]:
sample_table_apdist['ethanol_dilute_vol'].sum()

3860.0

In [9]:
stober.count_stock_vials(sample_table_apdist, 15000)

{'teos_count': 1, 'ammonia_count': 1, 'water_count': 1, 'ethanol_count': 1}

In [11]:
sample_table_rmse['ethanol_volume'].sum()/10000

0.8278928874181584

## Load Tools

In [12]:
syringe_10 = syringe.HTTPSyringe.from_config(2, "../../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(1, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_Hamiltonsyringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_Hamiltonsyringe.json")

Syringe name:  10cc_1
{'capacity': 10000, 'empty_position': 1830, 'full_position': 1217, 'name': '10cc_1'}
Syringe name:  1cc_1
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_1'}
Syringe name:  1cc_2_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_2_hamilton'}
Syringe name:  1cc_3_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_3_hamilton'}


In [13]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)

In [14]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3

## Load water syringe with water

In [15]:
water_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [16]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [17]:
water_syringe.set_pulsewidth(water_syringe.full_position+1, s = 10)

In [18]:
water_syringe.set_pulsewidth(water_syringe.full_position+150, s = 500)

In [19]:
water_syringe.load_syringe(740, water_syringe.full_position+150)

Loaded syringe, remaining volume 740 uL


## Load ammonia syringe

In [18]:
#ammonia_syringe.set_pulsewidth(1420, s = 10)

In [20]:
ammonia_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [21]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [22]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 10)

In [23]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+350, s = 500)

In [24]:
ammonia_syringe.load_syringe(400, ammonia_syringe.full_position+350)

Loaded syringe, remaining volume 400 uL


## Load TEOS syringe

In [25]:
teos_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [26]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [27]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+1, s = 100)

In [28]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+200, s = 200)

In [29]:
teos_syringe.load_syringe(690, teos_syringe.full_position+200)

Loaded syringe, remaining volume 690 uL


## load ethanol syringe

In [30]:
#jubilee.park_tool()

In [31]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


In [32]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1, s = 2000)


In [33]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+1, s = 10)

In [32]:
#mix_syringe.empty_position

In [34]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+100, s = 500)

In [35]:
mix_syringe.load_syringe(8100, mix_syringe.full_position+100)

Loaded syringe, remaining volume 8100 uL


## Load labware


In [43]:
samples_apdist = jubilee.load_labware('septavialrev1_44_holder_2000ul.json', 2)
samples_apdist.manual_offset([(19.3, 175.8), (132.3, 176.5), (133.2, 106.8)])

New manual offset applied to septavialrev1_44_holder_2000ul


In [44]:
samples_rmse = jubilee.load_labware('septavialrev1_44_holder_2000ul.json', 3)
samples_rmse.manual_offset([(160.2, 176.4),(270.9, 177.1),(272.1, 107.8)])

New manual offset applied to septavialrev1_44_holder_2000ul


In [45]:
stocks_main = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_main.manual_offset([(30.5, 266.0),  (120.5, 266.0), (120.5, 210.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [39]:
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

In [46]:
teos_stocks = [stocks_main[0]]
ammonia_stocks = [stocks_main[1]]
water_stocks = [stocks_main[2]]
ethanol_stocks_apdist = [stocks_main[3], stocks_main[4], stocks_main[5]]
rinse_stocks_apdist = [stocks_main[9], stocks_main[10], stocks_main[11]]

In [47]:
ethanol_stocks_rmse = [stocks_main[6], stocks_main[7], stocks_main[8]]
rinse_stocks_rmse = [stocks_main[9], stocks_main[10], stocks_main[11]]

## Check sample alignment

In [31]:
jubilee.park_tool()

In [61]:
jubilee.pickup_tool(mix_syringe)
for i, row in sample_table_rmse.iterrows():
    loc = samples_rmse[row['well']]
    jubilee.move_to(x = loc.x, y = loc.y)
    print(loc.y)
    jubilee.move_to(z = loc.top_+7)
    curr_well = row['well']
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

176.4


Is the tip centered over sample A1? 


176.58690525912615


Is the tip centered over sample A2? 


176.77381051825228


...

Is the tip centered over sample A8? 


162.53889037779504


Is the tip centered over sample B1? 


162.9127008960473


Is the tip centered over sample B3? 


In [55]:
jubilee.park_tool()


# Experiment


In [62]:
location_lookup_apdist = {row['uuid']:samples_apdist[row['well']] for i, row in sample_table_apdist.iterrows()}
location_lookup_rmse = {row['uuid']:samples_rmse[row['well']] for i, row in sample_table_rmse.iterrows()}

In [63]:
sample_table_apdist['ethanol_volume'].sum()

9769.506482027937

In [64]:
dilution_table_apdist = sample_table_apdist.copy()
dilution_table_rmse = sample_table_rmse.copy()

In [65]:
dilution_table_apdist['well'] = dilution_table_apdist['dilute_well']
dilution_table_rmse['well'] = dilution_table_rmse['dilute_well']

In [66]:
dilution_table_rmse

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,9cff952d-2d28-47b2-bb0b-d49602fd36d2,90.614116,189.664150,250.625458,1169.096276,1.193493,5.033963,0.006237,0.0,1700.0,B4,B4
1,1,60e54cc2-4d8e-4d0e-aab8-390bef44dcf8,228.684063,270.377199,470.888258,730.050480,0.472911,12.704280,0.015444,896.0,804.0,B5,B5
2,2,bde1a208-9fee-499e-843b-7866aa600a77,24.224999,484.500007,666.348728,524.926265,4.464286,1.345792,0.001622,0.0,1700.0,B6,B6
3,3,8f8ed9a5-2ea3-440f-93bf-01d9f6cbc33d,484.500007,24.224999,273.731449,917.543544,0.223214,26.915840,0.032983,1321.0,379.0,B8,B8
4,4,f93ed4f1-94fb-4806-835e-4975da39c452,135.707419,373.352663,489.220082,701.719836,0.796915,7.539070,0.009161,345.0,1355.0,C1,C1
5,5,99049d8c-98af-4240-ad63-7784477d7c02,379.921139,133.905259,221.684161,964.489441,0.284657,21.106082,0.025954,1216.0,484.0,C2,C2
6,6,a35e929b-49fd-4bed-a211-70524348fe8e,24.224999,212.324890,226.751481,1236.698630,4.464286,1.345792,0.001673,0.0,1700.0,C3,C3
7,7,069a5c5d-4bfd-419e-a96e-36215855466d,211.666209,484.500007,538.140465,465.693319,0.510933,11.758873,0.014169,831.0,869.0,C4,C4
8,8,627048de-ccfc-4b9e-998f-c83de2ac353f,199.575596,135.113642,701.939108,663.371654,0.541887,11.087193,0.013380,779.0,921.0,C5,C5
9,9,9f8223fc-c3be-4ca7-b9c9-55fa9c4daa60,484.500007,246.977225,63.183338,905.339430,0.223214,26.915840,0.033145,1321.0,379.0,C6,C6


In [67]:
location_lookup_dilute_apdist = {}

for i, row in dilution_table_apdist.iterrows():
    uuid_val = row['uuid']
    well = row['well']
    if well is not np.nan:
        well_loc = samples_apdist[well]
    else:
        well_loc = None

    location_lookup_dilute_apdist[uuid_val] = well_loc

In [68]:
location_lookup_dilute_rmse = {}

for i, row in dilution_table_rmse.iterrows():
    uuid_val = row['uuid']
    well = row['well']
    if well is not np.nan:
        well_loc = samples_rmse[well]
    else:
        well_loc = None

    location_lookup_dilute_rmse[uuid_val] = well_loc

In [69]:
dilution_table_rmse['ethanol_dilute_vol'].sum()

6709.0

## add ethanol

In [74]:
location_lookup_dilute_rmse

{'9cff952d-2d28-47b2-bb0b-d49602fd36d2': Well B4 form septavialrev1_44_holder_2000ul on slot 3,
 '60e54cc2-4d8e-4d0e-aab8-390bef44dcf8': Well B5 form septavialrev1_44_holder_2000ul on slot 3,
 'bde1a208-9fee-499e-843b-7866aa600a77': Well B6 form septavialrev1_44_holder_2000ul on slot 3,
 '8f8ed9a5-2ea3-440f-93bf-01d9f6cbc33d': Well B8 form septavialrev1_44_holder_2000ul on slot 3,
 'f93ed4f1-94fb-4806-835e-4975da39c452': Well C1 form septavialrev1_44_holder_2000ul on slot 3,
 '99049d8c-98af-4240-ad63-7784477d7c02': Well C2 form septavialrev1_44_holder_2000ul on slot 3,
 'a35e929b-49fd-4bed-a211-70524348fe8e': Well C3 form septavialrev1_44_holder_2000ul on slot 3,
 '069a5c5d-4bfd-419e-a96e-36215855466d': Well C4 form septavialrev1_44_holder_2000ul on slot 3,
 '627048de-ccfc-4b9e-998f-c83de2ac353f': Well C5 form septavialrev1_44_holder_2000ul on slot 3,
 '9f8223fc-c3be-4ca7-b9c9-55fa9c4daa60': Well C6 form septavialrev1_44_holder_2000ul on slot 3}

In [80]:
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, sample_table_apdist, location_lookup_apdist, 'ethanol_volume', ethanol_stocks_apdist[:2], stocks_usable_volume=15000, dwell_time = 10)

In [81]:

stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, sample_table_rmse, location_lookup_rmse, 'ethanol_volume', ethanol_stocks_rmse[:2], stocks_usable_volume=15000, dwell_time = 10)

In [82]:
#dispense remaining volume from etOH mix syringe
jubilee.pickup_tool(mix_syringe)
mix_syringe.dispense(mix_syringe.remaining_volume-1, trash[0])
logger.info('Emptied ethanol syringe into trash')
jubilee.park_tool()

In [83]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)

## Dispense water


In [84]:
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_apdist, location_lookup_apdist, 'water_volume', water_stocks, dwell_time = 7)

In [85]:
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_rmse, location_lookup_rmse, 'water_volume', water_stocks, dwell_time = 7)

## dispense Ammonia

In [86]:
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_apdist, location_lookup_apdist, 'ammonia_volume', ammonia_stocks)

In [87]:
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_rmse, location_lookup_rmse, 'ammonia_volume', ammonia_stocks)

In [88]:
rinse_stocks_rmse

[Well C2 form 20mLscintillation 12 Well Plate 18000 µL on slot 4,
 Well C3 form 20mLscintillation 12 Well Plate 18000 µL on slot 4,
 Well C4 form 20mLscintillation 12 Well Plate 18000 µL on slot 4]

## mix precursors

In [94]:
stober.first_mix(jubilee, mix_syringe, 2000, location_lookup_apdist, rinse_stocks_apdist, 5)

KeyboardInterrupt: 

In [ ]:
stober.first_mix(jubilee, mix_syringe, 2000, location_lookup_rmse, rinse_stocks_rmse, 5)

## Add TEOS and mix

In [91]:
teos_start_time = time.time()

In [92]:
stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_apdist, location_lookup_apdist, 'teos_volume', teos_stocks, mix_after=(2000, 5, rinse_stocks_apdist))

In [93]:
stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_rmse, location_lookup_rmse, 'teos_volume', teos_stocks, mix_after=(2000, 5, rinse_stocks_rmse))

### Dilution

In [96]:
jubilee.park_tool()

In [97]:
hrs5 = 60*60*5
while (time.time() - teos_start_time) < hrs5:
    time.sleep(60)

#dispese ethanol into dilution vials
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, dilution_table_apdist, location_lookup_dilute_apdist, 'ethanol_dilute_vol', ethanol_stocks_apdist[2:], stocks_usable_volume=15000, dwell_time = 10)

jubilee.pickup_tool(mix_syringe)
mix_syringe.dispense(mix_syringe.remaining_volume-1, trash[0])
logger.info('Emptied ethanol syringe into trash')
jubilee.park_tool()

jubilee.pickup_tool(mix_syringe)
rinse_vol = 2000
for i, row in dilution_table_apdist.iterrows():
    uuid_val = row['uuid']
    sample_well = location_lookup_apdist[uuid_val]
    dilute_well = location_lookup_dilute_apdist[uuid_val]
    if dilute_well is None:
        continue
    else:
        vol = row['sample_dilute_vol']
        
        mix_syringe.aspirate(vol, sample_well.bottom(+5))
        time.sleep(10)
        mix_syringe.dispense(vol, dilute_well.bottom(+5))
        time.sleep(10)
        logger.info(f'Transferred {vol} uL from {sample_well} to {dilute_well}')
    
        for stock in rinse_stocks_apdist:
            mix_syringe.mix(rinse_vol, 5, stock.bottom(+10), t_hold = 3, s_aspirate = 2000, s_dispense = 500)
    
        logger.info(f'Washed mix syringe in wash solutions {rinse_stocks_apdist}')

    

In [98]:
# dispese ethanol into dilution vials
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, dilution_table_rmse, location_lookup_dilute_rmse, 'ethanol_dilute_vol', ethanol_stocks_rmse[2:], stocks_usable_volume=15000, dwell_time = 10)

jubilee.pickup_tool(mix_syringe)
mix_syringe.dispense(mix_syringe.remaining_volume-1, trash[0])
logger.info('Emptied ethanol syringe into trash')
jubilee.park_tool()

jubilee.pickup_tool(mix_syringe)
rinse_vol = 2000
for i, row in dilution_table_rmse.iterrows():
    uuid_val = row['uuid']
    sample_well = location_lookup_rmse[uuid_val]
    dilute_well = location_lookup_dilute_rmse[uuid_val]
    if dilute_well is None:
        continue
    else:
        vol = row['sample_dilute_vol']
        
        mix_syringe.aspirate(vol, sample_well.bottom(+5))
        time.sleep(10)
        mix_syringe.dispense(vol, dilute_well.bottom(+5))
        time.sleep(10)
        logger.info(f'Transferred {vol} uL from {sample_well} to {dilute_well}')
    
        for stock in rinse_stocks_rmse:
            mix_syringe.mix(rinse_vol, 5, stock.bottom(+10), t_hold = 3, s_aspirate = 2000, s_dispense = 500)
    
        logger.info(f'Washed mix syringe in wash solutions {rinse_stocks_rmse}')

In [101]:
jubilee.park_tool()

In [91]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1)

In [92]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1)

In [93]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1)

In [99]:
sample_table_apdist

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,7c4d74ad-5362-4981-b7a5-52fee1cd2e48,100.446867,431.840514,39.876055,1127.836564,1.076662,5.580210,0.006942,0.0,1700.0,A1,B4
1,1,2b778460-0ea0-4134-8256-cc0950b1838f,47.743600,169.627361,480.552179,1002.076860,2.265169,2.652341,0.003253,0.0,1700.0,A2,B5
2,2,e73de4ac-ae2f-40ac-adc9-6f4acfbb4678,297.684084,187.260579,659.409887,555.645450,0.363296,16.537497,0.019903,1082.0,618.0,A3,B6
3,3,6e8ce412-9cdc-41b3-b8ae-fa08acb207cf,165.119200,457.338778,723.993653,353.548369,0.654965,9.173007,0.010973,587.0,1113.0,A4,B8
4,4,ec102405-f168-46a3-935d-2d24fdaa5b1f,83.489946,282.278352,27.288292,1306.943410,1.295333,4.638188,0.005802,0.0,1700.0,A5,C1
5,5,7bf7296f-8eae-4228-a615-3d7b8c142d89,74.110835,34.513714,48.689469,1542.685982,1.459265,4.117142,0.005183,0.0,1700.0,A6,C2
6,6,7d38b9e2-e3d3-494a-b43a-2997b6d7bf4c,91.456437,456.318141,63.359785,1088.865637,1.182501,5.080757,0.006310,0.0,1700.0,A7,C3
7,7,2252e0ad-3f6b-4234-9603-2420244f1080,230.246387,172.913187,526.767735,770.072691,0.469703,12.791073,0.015545,902.0,798.0,A8,C4
8,8,bf8528f0-4b3a-429d-bb11-da3c7d2c66f7,447.470939,365.846071,91.428678,795.254312,0.241686,24.858733,0.030505,1289.0,411.0,B1,C5
9,9,4c109404-def8-474c-a617-783ec03ec895,43.538614,94.802899,335.081280,1226.577207,2.483940,2.418738,0.002996,0.0,1700.0,B3,C6


In [100]:
sample_table_rmse

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,9cff952d-2d28-47b2-bb0b-d49602fd36d2,90.614116,189.664150,250.625458,1169.096276,1.193493,5.033963,0.006237,0.0,1700.0,A1,B4
1,1,60e54cc2-4d8e-4d0e-aab8-390bef44dcf8,228.684063,270.377199,470.888258,730.050480,0.472911,12.704280,0.015444,896.0,804.0,A2,B5
2,2,bde1a208-9fee-499e-843b-7866aa600a77,24.224999,484.500007,666.348728,524.926265,4.464286,1.345792,0.001622,0.0,1700.0,A3,B6
3,3,8f8ed9a5-2ea3-440f-93bf-01d9f6cbc33d,484.500007,24.224999,273.731449,917.543544,0.223214,26.915840,0.032983,1321.0,379.0,A4,B8
4,4,f93ed4f1-94fb-4806-835e-4975da39c452,135.707419,373.352663,489.220082,701.719836,0.796915,7.539070,0.009161,345.0,1355.0,A5,C1
5,5,99049d8c-98af-4240-ad63-7784477d7c02,379.921139,133.905259,221.684161,964.489441,0.284657,21.106082,0.025954,1216.0,484.0,A6,C2
6,6,a35e929b-49fd-4bed-a211-70524348fe8e,24.224999,212.324890,226.751481,1236.698630,4.464286,1.345792,0.001673,0.0,1700.0,A7,C3
7,7,069a5c5d-4bfd-419e-a96e-36215855466d,211.666209,484.500007,538.140465,465.693319,0.510933,11.758873,0.014169,831.0,869.0,A8,C4
8,8,627048de-ccfc-4b9e-998f-c83de2ac353f,199.575596,135.113642,701.939108,663.371654,0.541887,11.087193,0.013380,779.0,921.0,B1,C5
9,9,9f8223fc-c3be-4ca7-b9c9-55fa9c4daa60,484.500007,246.977225,63.183338,905.339430,0.223214,26.915840,0.033145,1321.0,379.0,B3,C6


In [151]:
dilution_table_apdist

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,ethanol_dilute_vol,sample_dilute_vol,well,dilute_well
0,0,bbd1bfff-7563-4360-b5b8-86a1deb3928f,91.455773,121.416734,255.000010,1232.127483,0.414916,14.480053,0.017314,995.0,705.0,C7,C7
1,1,5b9ac046-6ca6-4ebc-87f3-e272edaedb7b,140.351853,8.500000,255.000010,1296.148137,0.270366,22.221694,0.026673,1240.0,460.0,C8,C8
2,2,9f782755-5ca4-4466-a1f1-a87ef665ed22,255.000010,42.990994,255.000010,1147.008985,0.148810,40.373762,0.047775,1447.0,253.0,D1,D1
3,3,f8b2207f-b63c-44d9-8fe4-70639fdd3f5c,255.000010,53.976205,8.500000,1382.523785,0.148810,40.373762,0.049511,1447.0,253.0,D2,D2
4,4,61431cc9-9557-45af-82c4-9e7acb251717,78.545307,77.696470,193.651897,1350.106326,0.483115,12.435958,0.015077,879.0,821.0,D3,D3
5,5,e773e91e-cdd8-466c-be37-8ff86cb7f610,52.806463,59.674126,39.147849,1548.371562,0.718594,8.360767,0.010419,478.0,1222.0,D4,D4
6,6,4559d494-fcc2-4dfc-a4cf-a14d93eb0029,102.107158,177.169546,8.500000,1412.223295,0.371633,16.166470,0.019945,1068.0,632.0,D5,D5
7,7,78e5ec30-0b59-4536-a9e0-21b4bac433a8,67.497684,128.016714,8.500000,1495.985602,0.562189,10.686805,0.013285,744.0,956.0,D6,D6
8,8,e42bc4d2-0157-425c-aed0-4cdec425d9ab,120.342861,146.735320,171.336467,1261.585352,0.315319,19.053701,0.022952,1164.0,536.0,D7,D7
9,9,4133ded3-a2c0-4d14-b4fb-578358436a04,17.000000,159.878042,8.500000,1514.621958,2.232143,2.691584,0.003355,0.0,1700.0,D8,D8
